<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/RS_lecture_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=867256 sha256=c5de98e8d00183b205fb9b8ec463d24f9fd8716ce85eb07106b85c7d623f2b28
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
import numpy as np

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [ ]:
?fetch_movielens

In [ ]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [ ]:
train = movielens['train']
test = movielens['test']

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(random_state=42)
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

In [ ]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.44, test 0.13.


In [ ]:
model = LightFM(learning_rate=0.05, loss='warp', random_state=42)

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.61, test 0.22.


In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [ ]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        B
        D
        S
     Recommended:
        R
        R
        S
User 25
     Known positives:
        T
        T
        D
     Recommended:
        C
        S
        L
User 451
     Known positives:
        T
        B
        P
     Recommended:
        R
        G
        S


In [ ]:
known_positives = movielens['item_labels'][movielens['train'].tocsr()[2].indices]